# Install Dependencies

In this step, we install the packages required for our Dream Analyzer Agent.
These include:

google-generativeai → to use Gemini models

supabase → to store and retrieve dream history

python-dotenv → for secure environment variable handling

These tools allow our agent to generate text, analyze dreams, and store data safely.

In [66]:
!pip install google-generativeai supabase python-dotenv

# Environment Setup

Here we enter our Gemini API Key and Supabase credentials using getpass().
This keeps all keys safe and hidden.

We then configure Gemini so our notebook can communicate with the model.

In [57]:
from getpass import getpass

GEMINI_API_KEY = getpass("Enter your Gemini API Key: ")
SUPABASE_URL = getpass("Enter your Supabase URL: ") 
SUPABASE_KEY = getpass("Enter your Supabase API Key: ")

import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)

Enter your Gemini API Key:  ········
Enter your Supabase URL:  ········
Enter your Supabase API Key:  ········


In [67]:
from supabase import create_client, Client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Dream Ingest Agent
This agent allows the user to type their dream in natural language.

It has two parts:

1. get_dream_input()

* Takes the dream from the user

* Simple input function
  
     
2. clean_dream()

* Removes extra spaces

* Makes the text clean so AI can analyze it better

* This is the first step of the pipeline.


In [68]:
def get_dream_input():
    print("Write your dream:")
    dream = input()
    return dream.strip()

# Clean dream text
def clean_dream(text):
    return " ".join(text.split())

# Symbol Extraction Agent 
Dreams contain many symbols like:

* people
* objects
* animals
* places
* actions


This agent uses Gemini to:


* read the dream
* identify these symbols
* return them in clean JSON format


We also include:


* a JSON parser (to clean the model’s output)
* error handling
* a logging statement for observability


This makes our system more accurate and easier to debug.

In [69]:
import json

def extract_symbols(dream_text):
    prompt = f"""
    You are a Dream Symbol Extraction Agent.

    Extract symbols ONLY from this dream.

    Dream: {dream_text}

    Return STRICT JSON ONLY.
    No explanation.
    No extra text.
    No commentary.
    No backticks.

    The ONLY valid output:
    {{
      "people": [],
      "objects": [],
      "animals": [],
      "places": [],
      "actions": [],
      "other_symbols": []
    }}
    """

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)

    # HARD clean: remove code fences and random text
    cleaned = response.text.strip()
    cleaned = cleaned.replace("```json", "").replace("```", "").strip()

    # If JSON is NOT first, extract only the JSON part
    if "{" in cleaned:
        cleaned = cleaned[cleaned.index("{") : ]
    if "}" in cleaned:
        cleaned = cleaned[: cleaned.rindex("}") + 1]

    return cleaned

def parse_json_output(text):
    try:
        return json.loads(text)
    except:
        print("JSON error — printing raw output:")
        print(text)
        return None



# Emotion Agent

This agent identifies the emotional meaning of the dream.

It extracts:

* dominant emotion
* secondary emotions
* emotion intensity (1–10)

The result helps us understand the psychological context behind the dream.

In [70]:
def detect_emotions(dream_text):
    prompt = f"""
    You are an Emotion Detection Agent.

    Analyze the dream and detect:
    - dominant emotion
    - secondary emotions
    - intensity (1–10 scale)

    Dream: {dream_text}

    Return JSON only:
    {{
      "dominant": "",
      "secondary": [],
      "intensity": 0
    }}
    """

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    output = response.text.replace("```json", "").replace("```", "").strip()
    return json.loads(output)


# Interpretation Agent

This agent combines:

* the dream text
* the extracted symbols
* the detected emotions

Using this information, it generates:

1. Two psychological interpretations

2. One reflection question for self-growth

This makes the dream analysis deeply meaningful.

In [71]:
def interpret_dream(dream_text, symbols, emotions):
    prompt = f"""
    You are a Dream Interpretation Agent.

    Use:
    - Dream content
    - Extracted symbols: {symbols}
    - Detected emotions: {emotions}

    Provide:
    1) Two short interpretations
    2) One self-reflection question

    Format:
    Interpretation 1: ...
    Interpretation 2: ...
    Reflection Question: ...
    """

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text.strip()


# Embedding Generation

To enable memory and similarity search, we convert the dream into a vector embedding using:

models/text-embedding-004

Embeddings help the agent:

* remember past dreams
* find similar dreams
* answer questions based on stored memories

This step is essential for the Chat Agent.

In [72]:
def create_embedding(text):
    result = genai.embed_content(model="models/text-embedding-004", content=text)
    return result["embedding"]


# Save Dream to Supabase

Here we save all processed data into the Supabase database:

* dream text
* symbols
* emotions
* interpretation
* embedding

This becomes part of the user’s “dream memory bank”.

The saved data is later used for smart Q&A.

In [73]:
def save_to_supabase(dream_text, symbols, emotions, interpretation, embedding):
    data = {
        "dream_text": dream_text,
        "symbols": symbols,
        "emotions": emotions,
        "interpretation": interpretation,
        "embedding": embedding
    }

    response = supabase.table("dreams").insert(data).execute()
    return response


# Full Dream Analysis Pipeline

This function ties everything together.

Workflow:

1. Ask user for dream
2. Clean the text
3. Extract symbols
4. Detect emotions
5. Generate interpretation
6. Create embedding
7. Save everything into Supabase
8. Print results

After running this, your dream is fully processed and stored.

In [74]:
def run_full_pipeline():
    dream = get_dream_input()
    cleaned = clean_dream(dream)

    # 1. Extract symbols
    symbols = parse_json_output(extract_symbols(cleaned))

    # 2. Detect emotions
    emotions = detect_emotions(cleaned)

    # 3. Interpretation
    interpretation = interpret_dream(cleaned, symbols, emotions)

    # 4. Embedding
    embedding = create_embedding(cleaned)

    # 5. Save to Supabase
    save_to_supabase(cleaned, symbols, emotions, interpretation, embedding)

    print("\n🎉 DREAM ANALYZED & SAVED SUCCESSFULLY!")
    print("Symbols:", symbols)
    print("Emotions:", emotions)
    print("Interpretation:", interpretation)

# Run Dream Analysis

This cell executes the full pipeline.
You simply enter your dream, and the system handles everything automatically.

In [75]:
run_full_pipeline()

Write your dream:


 I saw snake 



🎉 DREAM ANALYZED & SAVED SUCCESSFULLY!
Symbols: {'people': [], 'objects': [], 'animals': ['snake'], 'places': [], 'actions': [], 'other_symbols': []}
Emotions: {'dominant': 'Fear', 'secondary': ['Surprise', 'Unease'], 'intensity': 7}
Interpretation: Interpretation 1:
The appearance of the snake, coupled with strong fear and unease, suggests you may be confronting a perceived threat or an unsettling, perhaps unexpected, challenge in your waking life. This dream could be highlighting a situation or person that evokes feelings of danger, betrayal, or a significant challenge you're finding difficult to navigate.

Interpretation 2:
Snakes are powerful symbols of transformation, hidden knowledge, or primal instincts. Your intense emotional reaction might indicate a resistance to a significant personal change unfolding, or a fear of confronting a deep, instinctual part of yourself or a truth that feels unsettling and unpredictable.

Reflection Question:
What specific situation or area in my 

# Embedding the User Question

When the user asks a question in chat, we convert their question into an embedding.

This helps us find which saved dream is most relevant to the question.

In [76]:
def embed_query(text):
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=text
    )
    return result["embedding"]


# Searching Dream Memories

This function performs a vector similarity search using Supabase.

It returns the top 3 most relevant dreams based on the user’s question.

This allows the agent to answer with context and memory.

In [77]:
def fetch_similar_dreams(query_embedding, match_count=3):
    response = supabase.rpc(
        "match_dreams",
        {
            "query_embedding": query_embedding,
            "match_count": match_count
        }
    ).execute()

    return response.data


# Building Chat Context

We combine the dreams retrieved by Supabase into a readable format.

This context is then fed into Gemini so the AI can give:

* meaningful
* personalized
* memory-aware

answers to the user's questions.

In [78]:
def build_context(dream_rows):
    context = ""
    for d in dream_rows:
        context += f"""
Dream: {d['dream_text']}
Symbols: {d['symbols']}
Emotions: {d['emotions']}
Interpretation: {d['interpretation']}
---
"""
    return context


# Chat Agent: Understanding the User's Question

This is the main Q&A function.

It uses:

* The user question

* The dream memories

* The context

* Gemini model

To produce:

* One meaning

* One emotional insight

* One reflection question

This allows the user to explore their dreams deeply.

In [79]:
def chat_about_dream(query):
    query_emb = embed_query(query)
    dream_rows = fetch_similar_dreams(query_emb, 3)
    context = build_context(dream_rows)

    prompt = f"""
    You are a Dream Chat Agent.

    User question: {query}
    Dream memories:
    {context}

    Give:
    - one meaning
    - one emotional insight
    - one reflection question
    """

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text



# Chat Loop — User Interaction

This is the interface for chatting with the agent.

Type **any question**, such as:

* “What does my dream mean?”

* “Why did I see a river?”

* “What emotional insight can you give?”

Type exit to stop.

In [80]:
# Chat Loop 
def chat_loop():
    print("Dream Chat Agent is ready! Type 'exit' to stop.\n")
    
    while True:
        query = input("You: ")

        if query.lower() in ["exit", "quit"]:
            print("Chat ended.")
            break

        answer = chat_about_dream(query)
        print("\nAI:", answer, "\n")

chat_loop()

Dream Chat Agent is ready! Type 'exit' to stop.



You:  Why did I saw snake 



AI: The snake in your dream is a powerful symbol that often appears when you are confronting significant aspects of your inner or outer world.

**Meaning:** Seeing a snake often points to a situation in your waking life that evokes strong feelings, representing either a perceived threat, a challenging situation you need to navigate, or a profound period of personal transformation and the uncovering of hidden truths.

**Emotional Insight:** Your dream highlights a range of intense emotions stirred by this symbol. It could be signaling a deep fear, unease, or surprise in response to a difficult circumstance, or it might be reflecting a fascinating mix of apprehension and curiosity about an unfolding change or an unknown path you are compelled to explore.

**Reflection Question:** What specific situation, challenge, or unfolding change in your life right now is evoking such strong feelings of fear, unease, or even a curious pull towards the unknown? 



You:  exit 



AI: Thank you for sharing your dreams with me. It's been a profound journey exploring their depths with you.

If you decide to return and explore more dreams, I'll be here.

Wishing you peace and clarity. 



You:  exit


Chat ended.
